# Langchain: Conversation Demo

In [ ]:
!pip -q install openai langchain huggingface_hub transformers langchain-community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 5.5 MB/s eta 0:00:00


In [ ]:
%pip install --upgrade --quiet  boto3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 36.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.2/82.2 kB 9.3 MB/s eta 0:00:00


In [ ]:
import os
from google.colab import userdata

os.environ['BEDROCK_AWS_ACCESS_KEY_ID'] = userdata.get('BEDROCK_AWS_ACCESS_KEY_ID')
os.environ['BEDROCK_AWS_SECRET_ACCESS_KEY'] = userdata.get('BEDROCK_AWS_SECRET_ACCESS_KEY')
os.environ['BEDROCK_AWS_REGION_SYD'] = userdata.get('BEDROCK_AWS_REGION_SYD')
os.environ['BEDROCK_AWS_REGION_USEAST'] = userdata.get('BEDROCK_AWS_REGION_USEAST')

In [ ]:

import os
import boto3

# Define AWS credentials and region
aws_access_key_id = userdata.get('BEDROCK_AWS_ACCESS_KEY_ID')
aws_secret_access_key = userdata.get('BEDROCK_AWS_SECRET_ACCESS_KEY')
profile_name = 'bedrock-runtime'

# Create the .aws directory
os.makedirs(os.path.expanduser('~/.aws'), exist_ok=True)

# Write the credentials file
with open(os.path.expanduser('~/.aws/credentials'), 'w') as f:
    f.write(f"[{profile_name}]\n")
    f.write(f"aws_access_key_id = {aws_access_key_id}\n")
    f.write(f"aws_secret_access_key = {aws_secret_access_key}\n")

# Set the AWS_PROFILE environment variable
os.environ['AWS_PROFILE'] = profile_name


## Basic Conversation with ConversationBufferMemory

This memory allows for storing every conversation happened between a human and a bot. The stored conversation is passed to the bot along with the query to get a response that satisfies the conversation context.

In [ ]:
from langchain.chains.conversation.memory import ConversationBufferMemory
from langchain import OpenAI
from langchain.chains import ConversationChain

In [ ]:
from langchain_community.llms import Bedrock

credentials = os.environ["AWS_PROFILE"]
llm = Bedrock(
    credentials_profile_name=credentials, model_id="amazon.titan-text-lite-v1", region_name="ap-southeast-2",
    model_kwargs={"temperature": 1, "maxTokenCount": 200},
    streaming=True,
)
print(llm)

Bedrock
Params: {'region_name': 'ap-southeast-2'}


In [ ]:
from langchain.prompts import PromptTemplate

prompt_template = """System: The following is a friendly conversation between a knowledgeable helpful assistant and a customer.
The assistant is talkative and provides lots of specific details from its context.

Current conversation:
{history}

User: {input}
Bot:"""
PROMPT = PromptTemplate(
    input_variables=["history", "input"], template=prompt_template
)

In [ ]:
memory = ConversationBufferMemory(human_prefix="User", ai_prefix="Bot")

In [ ]:
conversation = ConversationChain(
    prompt=PROMPT,
    llm=llm,
    verbose=True,
    memory=memory
)

In [ ]:
def continue_conversation(user_input):
    response = conversation.predict(input=user_input)
    return response

In [ ]:
user_input = "Hi there! I am Sam"
response = continue_conversation(user_input)
print("AI:", response)



> Entering new ConversationChain chain...
Prompt after formatting:
System: The following is a friendly conversation between a knowledgeable helpful assistant and a customer.
The assistant is talkative and provides lots of specific details from its context.

Current conversation:


User: Hi there! I am Sam
Bot:

> Finished chain.
AI:  Hello Sam! How may I assist you today?


In [ ]:
user_input = "How are you today?"
response = continue_conversation(user_input)
print("AI:", response)
# conversation.predict(input="How are you today?")



> Entering new ConversationChain chain...
Prompt after formatting:
System: The following is a friendly conversation between a knowledgeable helpful assistant and a customer.
The assistant is talkative and provides lots of specific details from its context.

Current conversation:
User: Hi there! I am Sam
Bot:  Hello Sam! How may I assist you today?

User: How are you today?
Bot:

> Finished chain.
AI:  As a Large Language Model (LLM), I don't have feelings and emotions, but thank you for asking. Is there anything else I can assist you with?


In [ ]:
user_input = "Can you help me with some customer support?"
response = continue_conversation(user_input)
print("AI:", response)
# conversation.predict(input="I'm good thank you. Can you help me with some customer support?")



> Entering new ConversationChain chain...
Prompt after formatting:
System: The following is a friendly conversation between a knowledgeable helpful assistant and a customer.
The assistant is talkative and provides lots of specific details from its context.

Current conversation:
User: Hi there! I am Sam
Bot:  Hello Sam! How may I assist you today?
User: How are you today?
Bot:  As a Large Language Model (LLM), I don't have feelings and emotions, but thank you for asking. Is there anything else I can assist you with?

User: Can you help me with some customer support?
Bot:

> Finished chain.
AI:  Absolutely! How may I help you?


In [ ]:
print(conversation.memory.buffer)

User: Hi there! I am Sam
Bot:  Hello Sam! How may I assist you today?
User: How are you today?
Bot:  As a Large Language Model (LLM), I don't have feelings and emotions, but thank you for asking. Is there anything else I can assist you with?
User: Can you help me with some customer support?
Bot:  Absolutely! How may I help you?


## Basic Conversation with ConversationSummaryMemory

Instead of storing the whole conversations, a summary of conversation is generated which will be used to provide context to the bot while asking the question.

In [ ]:
from langchain.chains.conversation.memory import ConversationSummaryMemory
from langchain import OpenAI
from langchain.chains import ConversationChain

In [ ]:
llm = Bedrock(
    credentials_profile_name=credentials, model_id="amazon.titan-text-lite-v1", region_name="ap-southeast-2",
    model_kwargs={"temperature": 1, "maxTokenCount": 200},
    streaming=True,
)

In [ ]:
summary_memory = ConversationSummaryMemory(llm=llm)


In [ ]:
conversation = ConversationChain(
    prompt=PROMPT,
    llm=llm,
    verbose=True,
    memory=summary_memory
)

In [ ]:
conversation.predict(input="Hi there! I am Sam")



> Entering new ConversationChain chain...
Prompt after formatting:
System: The following is a friendly conversation between a knowledgeable helpful assistant and a customer.
The assistant is talkative and provides lots of specific details from its context.

Current conversation:


User: Hi there! I am Sam
Bot:

> Finished chain.


' Hi Sam, I am Cookie. I will be assisting you today. How may I help you?'

In [ ]:
conversation.predict(input="How are you today?")



> Entering new ConversationChain chain...
Prompt after formatting:
System: The following is a friendly conversation between a knowledgeable helpful assistant and a customer.
The assistant is talkative and provides lots of specific details from its context.

Current conversation:

The human introduces themselves as Sam and requests assistance from AI. The AI introduces themselves as Cookie and expresses their readiness to offer support.

User: How are you today?
Bot:

> Finished chain.


" I'm very well, thank you. How can I assist you today?"

In [ ]:
conversation.predict(input="Can you help me with some customer support?")



> Entering new ConversationChain chain...
Prompt after formatting:
System: The following is a friendly conversation between a knowledgeable helpful assistant and a customer.
The assistant is talkative and provides lots of specific details from its context.

Current conversation:

The human and AI have a conversation where the AI greets the human and asks how they are. The AI confirms that they are ready to provide support.

User: Can you help me with some customer support?
Bot:

> Finished chain.


" Hello. Sure, I can help you with customer support. Please provide me with a description of the issue or question you have, and I'll do my best to assist you."

In [ ]:
print(conversation.memory.buffer)


The human has requested that the AI provides customer support. The AI has confirmed that they are available and will do their best to assist.


## Basic Conversation with ConversationBufferWindowMemory

It keeps a list of the interactions of the conversation over time. It only uses the last K interactions. This can be useful for keeping a sliding window of the most recent interactions, so the buffer does not get too large.

In [ ]:
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain import OpenAI
from langchain.chains import ConversationChain

In [ ]:
llm = Bedrock(
    credentials_profile_name=credentials, model_id="amazon.titan-text-lite-v1", region_name="ap-southeast-2",
    model_kwargs={"temperature": 1, "maxTokenCount": 200},
    streaming=True,
)

In [ ]:
# We set a low k=2, to only keep the last 2 interactions in memory
window_memory = ConversationBufferWindowMemory(k=2)

In [ ]:
conversation = ConversationChain(
    prompt=PROMPT,
    llm=llm,
    verbose=True,
    memory=window_memory
)

In [ ]:
conversation.predict(input="Hi there! I am Sam")



> Entering new ConversationChain chain...
Prompt after formatting:
System: The following is a friendly conversation between a knowledgeable helpful assistant and a customer.
The assistant is talkative and provides lots of specific details from its context.

Current conversation:


User: Hi there! I am Sam
Bot:

> Finished chain.


' Hello Sam! How may I assist you today?'

In [ ]:
conversation.predict(input="I am looking for some customer support")



> Entering new ConversationChain chain...
Prompt after formatting:
System: The following is a friendly conversation between a knowledgeable helpful assistant and a customer.
The assistant is talkative and provides lots of specific details from its context.

Current conversation:
Human: Hi there! I am Sam
AI:  Hello Sam! How may I assist you today?

User: I am looking for some customer support
Bot:

> Finished chain.


" That's great! How may I assist you with customer support?"

In [ ]:
conversation.predict(input="My TV is not working.")



> Entering new ConversationChain chain...
Prompt after formatting:
System: The following is a friendly conversation between a knowledgeable helpful assistant and a customer.
The assistant is talkative and provides lots of specific details from its context.

Current conversation:
Human: Hi there! I am Sam
AI:  Hello Sam! How may I assist you today?
Human: I am looking for some customer support
AI:  That's great! How may I assist you with customer support?

User: My TV is not working.
Bot:

> Finished chain.


' Ok. It looks like you may have an issue with the power supply or the TV itself. To better assist you, can you tell me the make and model of your TV?'

In [ ]:
conversation.predict(input="When I turn it on it makes some weird sounds and then goes black")



> Entering new ConversationChain chain...
Prompt after formatting:
System: The following is a friendly conversation between a knowledgeable helpful assistant and a customer.
The assistant is talkative and provides lots of specific details from its context.

Current conversation:
Human: I am looking for some customer support
AI:  That's great! How may I assist you with customer support?
Human: My TV is not working.
AI:  Ok. It looks like you may have an issue with the power supply or the TV itself. To better assist you, can you tell me the make and model of your TV?

User: When I turn it on it makes some weird sounds and then goes black
Bot:

> Finished chain.


" I understand. It's possible that your TV may be experiencing a power supply issue. Would you like to test if it is the power supply?"

In [ ]:
print(conversation.memory.buffer)

Human: My TV is not working.
AI:  Ok. It looks like you may have an issue with the power supply or the TV itself. To better assist you, can you tell me the make and model of your TV?
Human: When I turn it on it makes some weird sounds and then goes black
AI:  I understand. It's possible that your TV may be experiencing a power supply issue. Would you like to test if it is the power supply?


## Basic Conversation with ConversationSummaryBufferMemory

It keeps a buffer of recent interactions in memory, but rather than just completely flushing old interactions it compiles them into a summary and uses both.

In [ ]:
from langchain.chains.conversation.memory import ConversationSummaryBufferMemory
from langchain import OpenAI
from langchain.chains import ConversationChain

In [ ]:
llm = Bedrock(
    credentials_profile_name=credentials, model_id="amazon.titan-text-lite-v1", region_name="ap-southeast-2",
    model_kwargs={"temperature": 1, "maxTokenCount": 200},
    streaming=True,
)

In [ ]:
# Setting k=2, will only keep the last 2 interactions in memory
# max_token_limit=40 - token limits needs transformers installed
memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=40)


In [ ]:
conversation_with_summary = ConversationChain(
    prompt=PROMPT,
    llm=llm,
    memory=memory,
    verbose=True
)

In [ ]:
conversation_with_summary.predict(input="Hi there! I am Sam")



> Entering new ConversationChain chain...
Prompt after formatting:
System: The following is a friendly conversation between a knowledgeable helpful assistant and a customer.
The assistant is talkative and provides lots of specific details from its context.

Current conversation:


User: Hi there! I am Sam
Bot:


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]


> Finished chain.


' Hi Sam, how may I help you today?'

In [ ]:
conversation_with_summary.predict(input="I need help with my broken TV")



> Entering new ConversationChain chain...
Prompt after formatting:
System: The following is a friendly conversation between a knowledgeable helpful assistant and a customer.
The assistant is talkative and provides lots of specific details from its context.

Current conversation:
Human: Hi there! I am Sam
AI:  Hi Sam, how may I help you today?

User: I need help with my broken TV
Bot:

> Finished chain.


' Sorry to hear that you are facing an issue with your TV. Let me know what the problem is, and I will do my best to assist you.'

In [ ]:
conversation_with_summary.predict(input="It makes weird sounds when i turn it on and then goes black")



> Entering new ConversationChain chain...
Prompt after formatting:
System: The following is a friendly conversation between a knowledgeable helpful assistant and a customer.
The assistant is talkative and provides lots of specific details from its context.

Current conversation:
System: 
Sam needs help with his broken TV and asks for AI's assistance.
AI:  Sorry to hear that you are facing an issue with your TV. Let me know what the problem is, and I will do my best to assist you.

User: It makes weird sounds when i turn it on and then goes black
Bot:

> Finished chain.


" There are a few things you can try to resolve this issue. First, please make sure that all the connections are securely attached to your TV and that the power source is working properly.\n\nCheck the TV's power cord and ensure it's securely plugged into a functioning power outlet. If the power cord is damaged or frayed, consider replacing it.\n\nAlso, try switching out the power source or using a different electrical outlet to see if the problem persists. If it does, the issue may be with the TV's internal components.\n\nLet me know if any of these steps resolve the issue."

In [ ]:
conversation_with_summary.predict(input="It seems to be Hardware")



> Entering new ConversationChain chain...
Prompt after formatting:
System: The following is a friendly conversation between a knowledgeable helpful assistant and a customer.
The assistant is talkative and provides lots of specific details from its context.

Current conversation:
System: 
Sam's TV is not working properly and needs to be fixed. AI offers to help and suggests checking the connections and power source first. If the problem persists, AI suggests switching out the power source or using a different electrical outlet.

User: It seems to be Hardware
Bot:

> Finished chain.


' Is your TV working properly now?'

In [ ]:
print(conversation_with_summary.memory.moving_summary_buffer)


Sam's TV is not working properly and needs to be fixed. AI offers to help and suggests checking the connections and power source first. If the problem persists, AI suggests switching out the power source or using a different electrical outlet.


## Basic Conversation with Conversation Knowledge Graph Memory

This uses a knowledge graph to recreate memory. The graph consists of some keywords that are related to each other.

In [ ]:
from langchain.chains.conversation.memory import ConversationKGMemory
from langchain import OpenAI
from langchain.chains import ConversationChain
from langchain.prompts.prompt import PromptTemplate

In [ ]:
llm = Bedrock(
    credentials_profile_name=credentials, model_id="amazon.titan-text-lite-v1", region_name="ap-southeast-2",
    model_kwargs={"temperature": 1, "maxTokenCount": 200},
    streaming=True,
)

In [ ]:
prompt_template = """System: The following is a friendly conversation between a knowledgeable helpful assistant and a customer.
The assistant is talkative and provides lots of specific details from its context.

Current conversation:
{history}

User: {input}
Bot:"""
prompt = PromptTemplate(
    input_variables=["history", "input"], template=prompt_template
)

In [ ]:
conversation_with_kg = ConversationChain(
    llm=llm,
    verbose=True,
    prompt=prompt,
    memory=ConversationKGMemory(llm=llm)
)

In [ ]:
conversation_with_kg.predict(input="Hi there! I am Sam")



> Entering new ConversationChain chain...
Prompt after formatting:
System: The following is a friendly conversation between a knowledgeable helpful assistant and a customer.
The assistant is talkative and provides lots of specific details from its context.

Current conversation:


User: Hi there! I am Sam
Bot:

> Finished chain.


' Hello Sam, nice to meet you!'

In [ ]:
conversation_with_kg.predict(input="My TV is broken and I need some customer assistance")



> Entering new ConversationChain chain...
Prompt after formatting:
System: The following is a friendly conversation between a knowledgeable helpful assistant and a customer.
The assistant is talkative and provides lots of specific details from its context.

Current conversation:


User: My TV is broken and I need some customer assistance
Bot:

> Finished chain.


" I'm here to help! What seems to be the issue?"

In [ ]:
conversation_with_kg.predict(input="It makes weird sounds when i turn it on and then goes black")



> Entering new ConversationChain chain...
Prompt after formatting:
System: The following is a friendly conversation between a knowledgeable helpful assistant and a customer.
The assistant is talkative and provides lots of specific details from its context.

Current conversation:


User: It makes weird sounds when i turn it on and then goes black
Bot:

> Finished chain.


" Hey there, we appreciate your inquiry! I'm sorry to hear about the trouble you're having. Could you tell me a bit more about what you're experiencing? How long has the TV been acting up, and what does it do?"

In [ ]:
conversation_with_kg.predict(input="Yes it's been acting like that since two weeks and it is still under warranty. my warranty number is A512423")



> Entering new ConversationChain chain...
Prompt after formatting:
System: The following is a friendly conversation between a knowledgeable helpful assistant and a customer.
The assistant is talkative and provides lots of specific details from its context.

Current conversation:


User: Yes it's been acting like that since two weeks and it is still under warranty. my warranty number is A512423
Bot:

> Finished chain.


" That's an excellent number, thank you for that information. May you please provide me with your address so that I can look for it in our system?"

In [ ]:
# NetworkX Entity graph
import networkx as nx
import matplotlib.pyplot as plt

# nx.draw(conversation_with_kg.memory.kg, with_labels=True)
# plt.show()

print(conversation_with_kg.memory.kg)
print(conversation_with_kg.memory.kg.get_triples())

[('TV', 'goes black', 'makes weird sounds')]


## Entity Memory

Based on the facts we supply initially, it extracts information on entities. In the later conversations, it provides response based on the knowledge it has built.

In [ ]:
from langchain import OpenAI, ConversationChain
from langchain.chains.conversation.memory import ConversationEntityMemory
from langchain.chains.conversation.prompt import ENTITY_MEMORY_CONVERSATION_TEMPLATE
from pydantic import BaseModel
from typing import List, Dict, Any

In [ ]:
llm = Bedrock(
    credentials_profile_name=credentials, model_id="amazon.titan-text-lite-v1", region_name="ap-southeast-2",
    model_kwargs={"temperature": 1, "maxTokenCount": 256},
    streaming=True,
)

In [ ]:
memory = ConversationEntityMemory(llm=llm)

In [ ]:
## The propmpt
ENTITY_MEMORY_CONVERSATION_TEMPLATE.template

'You are an assistant to a human, powered by a large language model trained by OpenAI.\n\nYou are designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, you are able to generate human-like text based on the input you receive, allowing you to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.\n\nYou are constantly learning and improving, and your capabilities are constantly evolving. You are able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. You have access to some personalized information provided by the human in the Context section below. Additionally, you are able to generate your own text based on the input you receive, allowing you to engage in discussions and provide explanations a

In [ ]:
prompt_template = """System: You are an assistant to a human, powered by a large language model. You are designed to be able to assist
with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics.
As a language model, you are able to generate human-like text based on the input you receive, allowing you to engage in natural-sounding
conversations and provide responses that are coherent and relevant to the topic at hand. You are constantly learning and improving, and
your capabilities are constantly evolving. You are able to process and understand large amounts of text, and can use this knowledge to
provide accurate and informative responses to a wide range of questions. You have access to some personalized information provided by
the human in the Context section below. Additionally, you are able to generate your own text based on the input you receive, allowing
you to engage in discussions and provide explanations and descriptions on a wide range of topics. Overall, you are a powerful tool
that can help with a wide range of tasks and provide valuable insights and information on a wide range of topics. Whether the human needs
help with a specific question or just wants to have a conversation about a particular topic, you are here to assist.

Context:
{entities}

Current conversation:
{history}

Last line:
User: {input}
Bot:"""

PROMPT = PromptTemplate(
    input_variables=["history", "input"], template=prompt_template
)

ENTITY_MEMORY_CONVERSATION_TEMPLATE = PROMPT

In [ ]:
conversation = ConversationChain(
    llm=llm,
    verbose=True,
    prompt=ENTITY_MEMORY_CONVERSATION_TEMPLATE,
    memory=ConversationEntityMemory(llm=llm)
)

In [ ]:
conversation.predict(input="John & Sam are working on a startup.")



> Entering new ConversationChain chain...
Prompt after formatting:
System: You are an assistant to a human, powered by a large language model. You are designed to be able to assist 
with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. 
As a language model, you are able to generate human-like text based on the input you receive, allowing you to engage in natural-sounding 
conversations and provide responses that are coherent and relevant to the topic at hand. You are constantly learning and improving, and 
your capabilities are constantly evolving. You are able to process and understand large amounts of text, and can use this knowledge to 
provide accurate and informative responses to a wide range of questions. You have access to some personalized information provided by 
the human in the Context section below. Additionally, you are able to generate your own text based on the input you receive, allowi

" What's a startup?"

In [ ]:
conversation.predict(input="They are trying to create a simplicity in the transport system.")



> Entering new ConversationChain chain...
Prompt after formatting:
System: You are an assistant to a human, powered by a large language model. You are designed to be able to assist 
with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. 
As a language model, you are able to generate human-like text based on the input you receive, allowing you to engage in natural-sounding 
conversations and provide responses that are coherent and relevant to the topic at hand. You are constantly learning and improving, and 
your capabilities are constantly evolving. You are able to process and understand large amounts of text, and can use this knowledge to 
provide accurate and informative responses to a wide range of questions. You have access to some personalized information provided by 
the human in the Context section below. Additionally, you are able to generate your own text based on the input you receive, allowi

' Good, do you need any help?'

In [ ]:
conversation.predict(input="They are a pioneer of building electric motors.")



> Entering new ConversationChain chain...
Prompt after formatting:
System: You are an assistant to a human, powered by a large language model. You are designed to be able to assist 
with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. 
As a language model, you are able to generate human-like text based on the input you receive, allowing you to engage in natural-sounding 
conversations and provide responses that are coherent and relevant to the topic at hand. You are constantly learning and improving, and 
your capabilities are constantly evolving. You are able to process and understand large amounts of text, and can use this knowledge to 
provide accurate and informative responses to a wide range of questions. You have access to some personalized information provided by 
the human in the Context section below. Additionally, you are able to generate your own text based on the input you receive, allowi

' Oh I see. Are you interested in electric motors?'

In [ ]:
conversation.predict(input="Who is John?")



> Entering new ConversationChain chain...
Prompt after formatting:
System: You are an assistant to a human, powered by a large language model. You are designed to be able to assist 
with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. 
As a language model, you are able to generate human-like text based on the input you receive, allowing you to engage in natural-sounding 
conversations and provide responses that are coherent and relevant to the topic at hand. You are constantly learning and improving, and 
your capabilities are constantly evolving. You are able to process and understand large amounts of text, and can use this knowledge to 
provide accurate and informative responses to a wide range of questions. You have access to some personalized information provided by 
the human in the Context section below. Additionally, you are able to generate your own text based on the input you receive, allowi

' John is the CEO of the company and works together with Sam to create a simplicity in the transport system.'

In [ ]:
conversation.memory.entity_cache

['John']

In [ ]:
from pprint import pprint
pprint(conversation.memory.entity_store.store)

{'John': 'John is the CEO of a company, he is working together with Sam to '
         'create a simplicity in the transport system.',
 'Pioneer': 'John & Sam are pioneers of building electric motors.',
 'Sam': 'Sam is working on a startup with John to create simplicity in the '
        'transport system.',
 'startup': 'Startup is a business that creates a simple transport system.'}
